In [1]:
import logging
import yaml

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio

# import palpy

from lsst.ts import salobj

from lsst.ts.standardscripts.auxtel.attcs import ATTCS

from lsst.ts.idl.enums import ATPtg

%matplotlib inline

# Start up

In [2]:
from astropy.utils import iers
iers.conf.auto_download = False

In [3]:
d = salobj.Domain()

In [4]:
atmcs = salobj.Remote(d, "ATMCS")
atptg = salobj.Remote(d, "ATPtg")
ataos = salobj.Remote(d, "ATAOS")
atpne = salobj.Remote(d, "ATPneumatics")
athex = salobj.Remote(d, "ATHexapod")
atdome = salobj.Remote(d, "ATDome", index=1)
atdomtraj = salobj.Remote(d, "ATDomeTrajectory")

In [5]:
await asyncio.gather(atmcs.start_task, 
                     atptg.start_task,
                     ataos.start_task,
                     atpne.start_task,
                     athex.start_task,
                     atdome.start_task,
                     atdomtraj.start_task)

AckCmdReader(ATPneumatics, 0, ackcmd) falling behind; read 10 messages
Could not read historical data in 60.66 sec


[None, None, None, None, None, None, None]

## ATMCS

In [23]:
await asyncio.sleep(5)

RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(ATMCS, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(ATPneumatics, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, mount_Nasmyth_Encoders) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, mount_AzEl_Encoders) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, measuredTorque) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0

In [24]:
await salobj.set_summary_state(atmcs, salobj.State.ENABLED, timeout=120)

[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

## ATPtg

In [13]:
await atptg.tel_timeAndDate.next(flush=True, timeout=5)
await asyncio.sleep(2.)
await salobj.set_summary_state(atptg, salobj.State.ENABLED)

RemoteTelemetry(ATPneumatics, 0, m1AirPressure) falling behind; read 28 messages
RemoteTelemetry(ATMCS, 0, mount_AzEl_Encoders) falling behind; read 28 messages
RemoteEvent(ATPneumatics, 0, heartbeat) falling behind; read 28 messages


[<State.ENABLED: 2>]

In [ ]:
for i in range(10):
    time = await atptg.tel_timeAndDate.next(flush=True, timeout=5)
    print(time)

In [34]:
await atptg.cmd_pointLoadModel.set_start(pointingFile='/home/saluser/repos/ts_pointing_common/install/data/at.mod')

RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 100 messages


In [44]:
ptm = await atptg.evt_pointingModel.next(flush=False, timeout=5)

In [45]:
ptm.pointingModelTermNames, ptm.pointingModelTermValues

('IA,IE,NPAE,CA,AW,AN',
 '+2478.5196   279.49890,-1712.5127     9.55716,+336.2051   509.91209,-522.5508   570.42316,+249.6647    10.11987,+27.5525    10.21499')

In [ ]:
await salobj.set_summary_state(atptg, salobj.State.STANDBY)

## ATAOS

In [15]:
await ataos.evt_heartbeat.next(flush=True)
await ataos.cmd_setLogLevel.set_start(level=logging.DEBUG, timeout=10)
# await salobj.set_summary_state(ataos, salobj.State.STANDBY)
await salobj.set_summary_state(ataos, salobj.State.ENABLED, settingsToApply="measured_20190908.yaml")

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [ ]:
await ataos.cmd_enableCorrection.set_start(m1=True, timeout=10)

In [9]:
await ataos.cmd_disableCorrection.set_start(m1=True, timeout=10)

RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 80 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 16 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 16 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 16 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 16 messages
RemoteEvent(ATMCS, 0, heartbeat) falling behind; read 16 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 16 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 16 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 16 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 80 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 12 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 16 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 16 messages
RemoteTelemetry(ATPtg, 0, guidingAndOf

In [ ]:
await salobj.set_summary_state(ataos, salobj.STANDBY)

## ATPneumatics

In [25]:
await salobj.set_summary_state(atpne, salobj.State.ENABLED)

RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 100 messages
RemoteEvent(ATPneumatics, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(ATMCS, 0, heartbeat) falling behind; read 100 messages


[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [ ]:
await atpne.cmd_openM1Cover.start(timeout=30)

## ATDome

In [79]:
await asyncio.sleep(5)
await salobj.set_summary_state(atdome, salobj.State.ENABLED, settingsToApply="test.yaml")

RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteEvent(ATMCS, 0, heartbeat) falling behind; read 79 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 79 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 78 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 78 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 78 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 78 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 78 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 79 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 79 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 79 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 79 messages
RemoteTelemetry(ATMCS, 0, mo

[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [89]:
await atdome.cmd_moveAzimuth.set_start(azimuth=90., timeout=30)

RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
RemoteEvent(ATPtg, 0, weatherDataApplied) falling behind; read 44 messages


In [86]:
await atdome.cmd_homeAzimuth.start()

In [87]:
await atdome.cmd_openShutter.start()

RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 80 messages


In [82]:
await atdome.cmd_stopMotion.start()

In [75]:
err = await atdome.evt_errorCode.next(flush=False, timeout=1)
err = atdome.evt_errorCode.get()

TimeoutError: 

In [52]:
err.errorReport

'Could not open connection to host=192.168.223.14, port=17310: '

In [85]:
await atdome.cmd_moveShutterMainDoor.set_start(open=True, timeout=10)

RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 74 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 73 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 73 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 73 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 74 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 74 messages
RemoteTelemetry(ATPneumatics, 0, m1AirPressure) falling behind; read 74 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 73 messages
RemoteTelemetry(ATPtg, 0, currentTimesToLimits) falling behind; read 14 messages


In [78]:
await asyncio.sleep(5)
await salobj.set_summary_state(atdome, salobj.State.STANDBY)

RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTimesToLimits) falling behind; read 20 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 100 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, mount_Nasmyth_Encoders) falling behind; re

[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]

## ATDomeTrajectory

In [ ]:
await salobj.set_summary_state(atdomtraj, salobj.State.ENABLED)

## ATHexapod

In [ ]:
await salobj.set_summary_state(athex, salobj.State.ENABLED, settingsToApply="Default1")

In [ ]:
tel = await athex.evt_positionUpdate.next(flush=False, timeout=2)

In [ ]:
tel.positionX, tel.positionY, tel.positionZ, tel.positionU, tel.positionV

In [ ]:
await athex.cmd_moveToPosition.set_start(x=-4., y=1.,z=0.35, u=0.35, v=0.22)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=-15., az=0., num=1)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=15., az=0., num=1)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=0., az=-15., num=1)

In [ ]:
await salobj.set_summary_state(athex, salobj.State.STANDBY)

## Slewing/Tracking

In [ ]:
attcs = ATTCS(atmcs=atmcs, 
              atptg=atptg, 
              ataos=ataos, 
              atpneumatics=atpne, 
              athexapod=athex, 
              atdome=atdome, 
              atdometrajectory=atdomtraj, 
              check={"atpneumatics": False, "athexapod": False, "atdome": True, "atdometrajectory": True})

In [ ]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [ ]:
ra = Angle("16:57:59", unit=u.hour)
dec = Angle("-22:21:29", unit=u.deg)
target_name="Jupiter"
radec = ICRS(ra, dec)

In [ ]:

radec = ICRS(ra, dec)

# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
print(curr_time_atptg)
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print("slew...")
# await atmcs.cmd_startTracking.start(timeout=10)
await attcs.slew(ra=radec.ra.hour, 
                 dec=radec.dec.deg,
                 rotPA=180.-alt_az.alt.deg,
                 rot_frame=ATPtg.RotFrame.FIXED,
                 rot_mode=ATPtg.RotMode.FIELD)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=30., az=0.,num=1)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=-250.)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=0., az=-30., num=1)

In [ ]:
await atptg.cmd_offsetAbsorb.set_start(num=2,timeout=10)

In [ ]:
await atptg.cmd_exitControl.start()

In [ ]:
await atmcs.cmd_startTracking.start(timeout=30)

In [ ]:
# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
print(curr_time_atptg)
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print("slew...")
# await atmcs.cmd_startTracking.start(timeout=10)
await atptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt_az.alt.deg,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

In [ ]:
off = await atptg.tel_guidingAndOffsets.next(flush=True, timeout=5)

In [ ]:
off.userOffsetDec, off.handsetOffsetDec, off.userOffsetRA, off.handsetOffsetRA

# alf Sco

In [ ]:
ra = Angle("16:29:24.45970"  , unit=u.hour)
dec = Angle("-26:25:55.2094", unit=u.deg)
target_name="alf Sco"
radec = ICRS(ra, dec)



In [ ]:
# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
print(curr_time_atptg)
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print("slew...")
# await atmcs.cmd_startTracking.start(timeout=10)
await atptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt_az.alt.deg,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

## kap Sco

In [ ]:
ra=
dec=

In [ ]:
ra = Angle("17:42:29.27520" , unit=u.hour)
dec = Angle("-39:01:47.9391", unit=u.deg)
target_name="Kap Sco"
radec = ICRS(ra, dec)

In [ ]:
# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
print(curr_time_atptg)
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print("slew...")
# await atmcs.cmd_startTracking.start(timeout=10)
await atptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt_az.alt.deg,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

In [ ]:
await atptg.cmd_stopTracking.start(timeout=10)

In [ ]:
# off.userOffsetDec, off.handsetOffsetDec, off.userOffsetRA, off.handsetOffsetRA
await atptg.cmd_offsetRADec.set_start(type=1, 
                                      off1=off.userOffsetRA+off.handsetOffsetRA,
                                      off2=off.userOffsetDec+off.handsetOffsetDec,
                                     num=0)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=0., az=-250., num=1)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=0., az=10., num=1)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=-15., az=0., num=1)

# GRID

In [ ]:
async def wait_center():
    while True:
        opt = input("Center telescope and choose action (press ?<enter>, for list of actions): ")
        
        if opt == 'n':
            print("Next point in the grid...")
            return 1
        elif opt == 'p':
            print("Previous point in the grid...")
            return -1
        elif opt == 't':
            print("Stopping test...")
            return 0
        elif opt == '?':
            print("""Options are:
            n - Go to next point in the grid.
            p - Go to previous point in the grid.
            t - Terminate test.
            """)
        else:
            print("Next point in the grid...")
            return 1

In [ ]:
dalt_grid = np.arange(-0.25, +0.35, 0.1)+0.6
daz_grid = np.arange(-0.25, +0.35, 0.1)-0.6

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=0.6*60.*60., 
                                     az=-0.6*60*60, 
                                         num=0)

In [ ]:
grid_alt = np.zeros(len(dalt_grid)*len(daz_grid))
grid_az = np.zeros(len(dalt_grid)*len(daz_grid))

for i in range(len(daz_grid)):
    grid_alt[i*len(dalt_grid):(i+1)*len(dalt_grid)] += dalt_grid[::(-1)**i]
    grid_az[i*len(dalt_grid):(i+1)*len(dalt_grid)] += daz_grid[i]

In [ ]:
plt.plot(grid_alt, grid_az, ':')
plt.plot(grid_alt, grid_az, '|')
plt.plot(grid_alt[0], grid_az[0], 'bo')
plt.plot(grid_alt[-1], grid_az[-1], 'go')

In [ ]:
start_from=0

In [ ]:
print(f"Grid has {len(grid_alt)} pointings...")
print(f"Starting from {start_from}")

i = start_from
stop_at = 0

while i < len(grid_alt):
    
    print(f"GRID[{i}]: {grid_alt[i]} x {grid_az[i]}")

    await atptg.cmd_offsetAzEl.set_start(el=grid_alt[i]*60.*60., 
                                         az=grid_az[i]*60*60, 
                                         num=0)
    
    ret_val = await wait_center()

    if ret_val == 0:
        break
        
    i += ret_val
    
    if i < 0:
        print(f"Unwrapping index! (i={i})")
        i = 0

In [ ]:
i=36
await atptg.cmd_offsetAzEl.set_start(el=grid_alt[i]*60.*60., 
                                     az=grid_az[i]*60*60, 
                                     num=0)



# Parking

In [10]:
await atpne.cmd_closeM1Cover.start(timeout=30)

RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 46 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 46 messages
RemoteEvent(ATMCS, 0, heartbeat) falling behind; read 46 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 46 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 46 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 45 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 46 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 46 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 46 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 46 messages
RemoteEvent(ATDome, 1, heartbeat) f

In [11]:
await salobj.set_summary_state(atdomtraj, salobj.State.STANDBY)

RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 56 messages
RemoteTelemetry(ATPtg, 0, currentTimesToLimits) falling behind; read 11 messages


[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]

In [13]:
await asyncio.sleep(2.5)
atptg.evt_inPosition.flush()
await atptg.cmd_azElTarget.set_start(azDegs=0., elDegs=80.)
while True:
    inp = await atptg.evt_inPosition.next(flush=False, timeout=120.)
    if inp.inPosition:
        break

RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteEvent(ATMCS, 0, heartbeat) falling behind; read 21 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 21 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 21 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 21 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 20 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 21 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 21 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 20 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 21 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 21 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_moun

KeyboardInterrupt: 

In [14]:
await atdome.cmd_closeShutter.start(timeout=30)

AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 55 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 55 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 11 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 11 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 11 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 11 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 11 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 11 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 11 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behin

ERROR! Session/line number was not unique in database. History logging moved to new session 122


In [17]:
await atptg.cmd_stopTracking.start()

RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 39 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 39 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 39 messages


TimeoutError: 

RemoteEvent(ATAOS, 0, logMessage) falling behind; read 21 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 39 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 39 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 40 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 40 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages


falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [18]:
await atdome.cmd_moveAzimuth.set_start(azimuth=200.)

RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTimesToLimits) falling behind; read 46 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 100 messages


In [19]:
await asyncio.gather(salobj.set_summary_state(atmcs, salobj.State.STANDBY),
                     salobj.set_summary_state(atptg, salobj.State.STANDBY),
                     salobj.set_summary_state(atdome, salobj.State.STANDBY),
                     salobj.set_summary_state(athex, salobj.State.STANDBY),
                     salobj.set_summary_state(atpne, salobj.State.STANDBY),
                     salobj.set_summary_state(ataos, salobj.State.STANDBY))

RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 76 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


[[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>],
 [<State.FAULT: 3>, <State.STANDBY: 5>],
 [<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>],
 [<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>],
 [<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>],
 [<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]]

In [21]:
await asyncio.sleep(5)
await salobj.set_summary_state(atdome, salobj.State.DISABLED)

RemoteEvent(ATPneumatics, 0, heartbeat) falling behind; read 12 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 56 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 11 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 11 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 11 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 11 messages
RemoteEvent(ATMCS, 0, heartbeat) falling behind; read 11 messages


[<State.STANDBY: 5>, <State.DISABLED: 1>]

In [ ]:
print(salobj.State(ss.summaryState))

In [ ]:
await asyncio.sleep(10.)

In [ ]:
await salobj.set_summary_state(atmcs, salobj.State.STANDBY, timeout=120)

In [ ]:
def mainValveCallback(data):
    print(data.state)

In [ ]:
atpne.evt_mainValveState.callback = mainValveCallback

In [ ]:
atpne.evt_mainValveState.callback = None

In [ ]:
await asyncio.sleep(10.)
# mv = atpne.evt_mainValveState.get()

In [ ]:
mv.state

In [ ]:
await atpne.cmd_closeMasterAirSupply.start(timeout=5)

In [ ]:
mv = await atpne.evt_mainValveState.next(flush=False, timeout=2)

In [ ]:
print(mv)

In [ ]:
mv.state, mv.private_sndStamp

In [ ]:
mv = atpne.evt_mainValveState.get()

In [ ]:
print(mv)

In [ ]:
mv.state, mv.private_sndStamp

In [ ]:
await atpne.cmd_openMasterAirSupply.start(timeout=5)

In [ ]:
mv = await atpne.evt_instrumentState.next(flush=False, timeout=2)

In [ ]:
mv.state

In [ ]:
await atpne.cmd_openInstrumentAirValve.start(timeout=5)

In [ ]:
try:
    await atpne.cmd_m1OpenAirValve.start(timeout=10)
except salobj.AckError as e:
    if e.ackcmd.ack == salobj.SalRetCode.CMD_NOPERM:
        print(e)
    else:
        raise e

In [ ]:
await asyncio.sleep(10.)

In [ ]:
await atdomtraj.cmd_start.start()

In [ ]:
await asyncio.sleep(1.)
atpne.evt_m1State.flush()
await atpne.cmd_m1OpenAirValve.start(timeout=10)
m1state = await atpne.evt_m1State.next(flush=False, timeout=10)
print(m1state.state)

In [ ]:
await asyncio.sleep(1.)
atpne.evt_m1State.flush()
await atpne.cmd_m1CloseAirValve.start(timeout=10)
m1state = await atpne.evt_m1State.next(flush=False, timeout=10)
print(m1state.state)

In [ ]:
await asyncio.sleep(5.)
atpne.evt_instrumentState.flush()
await atpne.cmd_openInstrumentAirValve.start(timeout=10)
m1state = await atpne.evt_instrumentState.next(flush=False, timeout=10)
print(m1state.state)
await asyncio.sleep(5.)

In [ ]:
await asyncio.sleep(5.)
atpne.evt_instrumentState.flush()
await atpne.cmd_closeInstrumentAirValve.start(timeout=10)
m1state = await atpne.evt_instrumentState.next(flush=False, timeout=10)
print(m1state.state)
await asyncio.sleep(5.)

In [ ]:
await atpne.cmd_openMasterAirSupply.start()